In [1]:
import json
from pathlib import Path

import pandas as pd
import requests

from config import Config

In [2]:
base_dir = Path(".").resolve()
data_dir = base_dir / "data"

In [3]:
headers = {
    "Accept": "text/plain",
    "Authorization": f"ApiKey {Config.APIKEY}",
    "Content-Type": "application/json",
}

metadata = {
    "editState": "edit_state",
    "isObsolete": "obsolete",
    "name": "name",
    "description": "description",
    "lastUser": "updated_by",
    "lastDate": "updated_on",
    "tag": "tag",
    "id": "id",
    "rrg": "range",
    "cli": "credential",
    "ps": "paying_society",
    "cp": "refundable",
    "rat": "rate",
    "prv": "provider",
    "hot": "hotel",
    "dest": "destination",
    "mrk": "market",
    "mel": "meal",
    "cid": "check_in",
    "bod": "booking_date",
    "rel": "max_release",
    "isDynamicCommission": "dynamic_commission",
    "dow": "days_of_week",
    "age": "age",
    "room": "room",
    "non": "num_of_nights",
    "hou": "hours",
    "level_mapping": {"t": "level", "l": "list", "f": "from", "u": "to"},
}

In [4]:
url = "http://distributor.rules.api.pro.logitravel.internal/api/organizations/lgt/agencies/products/hot/level_closes/"

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    data = json.loads(
        json.dumps(data), object_hook=lambda obj: {**obj.pop("lvl", {}), **obj}
    )
    rules = data.get("rules", [])

    for rule in rules:
        rrg = rule.get("rrg")
        if rrg is None:
            rule["rrg"] = {"f": 0, "t": 0, "u": 0}

        for key in list(rule.keys()):
            if key in metadata:
                rule[metadata[key]] = rule.pop(key)

        level_mapping = metadata["level_mapping"]
        for key, value in rule.items():
            if isinstance(value, dict):
                for k in list(value.keys()):
                    if k in level_mapping:
                        value[level_mapping[k]] = value.pop(k)

In [8]:
df_rules = pd.json_normalize(rules, sep="_")
df_rules = df_rules[
    [
        "id",
        "name",
        "description",
        "tag",
        "credential_level",
        "credential_list",
        "provider_level",
        "provider_list",
        "hotel_level",
        "hotel_list",
        "destination_level",
        "destination_list",
        "rate",
        "refundable",
        "market_level",
        "market_list",
        "meal_level",
        "meal_list",
        "check_in_level",
        "check_in_from",
        "check_in_to",
        "booking_date_level",
        "booking_date_from",
        "booking_date_to",
        "range_level",
        "range_from",
        "range_to",
        "max_release",
        "days_of_week_level",
        "days_of_week_list",
        "age",
        "room_level",
        "room_list",
        "num_of_nights_level",
        "num_of_nights_list",
        "hours_level",
        "hours_list",
        "dynamic_commission",
        "obsolete"
    ]
]

In [ ]:
with pd.option_context("display.max_columns", None):
    display(df_rules.head())

In [ ]:
# Set filter to level 1 rules
filter_lvl1 = (
    (df_rules["tag"] == 1)
    & (df_rules["credential_level"] == 1)
    & (df_rules["provider_level"] == 1)
    & (df_rules["hotel_level"] == 1)
    & (df_rules["destination_level"] == 0)
    & (df_rules["rate"] == 0)
    & (df_rules["refundable"] == 0)
    & (df_rules["market_level"] == 0)
    & (df_rules["meal_level"] == 0)
    & (df_rules["check_in_level"] == 0)
    & (df_rules["booking_date_level"] == 0)
    & (df_rules["range_level"] == 0)
    & (df_rules["max_release"] == 0)
    & (df_rules["days_of_week_level"] == 0)
    & (df_rules["age"] == 0)
    & (df_rules["room_level"] == 0)
    & (df_rules["num_of_nights_level"] == 0)
    & (df_rules["hours_level"] == 0)
)

In [ ]:
df_rules_lvl1 = df_rules[filter_lvl1].copy()
df_rules_lvl1.shape

In [ ]:
df_rules_lvl1["credential_count"] = df_rules_lvl1["credential_list"].apply(len)
df_rules_lvl1.insert(6, "credential_count", df_rules_lvl1.pop("credential_count"))

In [ ]:
df_rules_lvl1.to_csv("df_rules_lvl1.csv", index=False)

In [ ]:
df_529117 = df_rules_lvl1[df_rules_lvl1["id"] == "529117"]

In [ ]:
df_529117 = df_529117.explode("credential_list")

In [ ]:
df_529117.shape

In [10]:
df_rules_obsolete = df_rules[df_rules["obsolete"]].copy()
df_rules_obsolete.shape

(65, 39)

In [11]:
df_rules_obsolete.to_csv("df_rules_obsolete.csv", index=False)

In [12]:
df_rules.to_csv("df_rules.csv", index=False)